In [9]:
%store -r log_text

In [11]:
from micropython_magic.memoryinfo import MemoryInfoList

mi_list = MemoryInfoList(show_free=False)
print(f"Loaded and parsed {len(mi_list.parse_log(log_text))} memory_info records from the log file")

Loaded and parsed 323 memory_info records from the log file


In [21]:
import bqplot.pyplot as plt
import numpy as np


def meminfo_plot(
    mi_list: MemoryInfoList,
    title: str = "Memory Info",
    used: bool = False,
    one_blocks: bool = False,
    two_blocks: bool = False,
    max_block_size: bool = False,
    max_free_size: bool = False,
    stack_used: bool = False,
    stack_total: bool = False,
    name_step: int = 5,
    legend_location: str = "left",
    display_toolbar=True,
):
    """
    Plot the memory info data from a log file.
    use the parameters to select the data to be plotted.
    """
    # left axis is the free memory
    free_mem = [i.free for i in mi_list]

    # right axis is configurable
    data_l = np.array([[i.free for i in mi_list]])
    data_r = np.array([[i.free for i in mi_list]])
    labels_l = ["free <"]
    labels_r = []
    if used:
        data_l = np.vstack([data_l, [i.used for i in mi_list]])
        labels_l.append("used <")
    if one_blocks:
        data_r = np.vstack([data_r, [i.one_blocks for i in mi_list]])
        labels_r.append("1-blocks >")
    if two_blocks:
        data_r = np.vstack([data_r, [i.two_blocks for i in mi_list]])
        labels_r.append("2-blocks >")
    if max_block_size:
        data_r = np.vstack([data_r, [i.max_block_size for i in mi_list]])
        labels_r.append("max block size >")
    if max_free_size:
        data_r = np.vstack([data_r, [i.max_free_size for i in mi_list]])
        labels_r.append("max free size >")
    if stack_used:
        data_r = np.vstack([data_r, [i.stack_used for i in mi_list]])
        labels_r.append("stack used >")
    if stack_total:
        data_r = np.vstack([data_r, [i.stack_total for i in mi_list]])
        labels_r.append("stack total >")
    # throw away the first row
    data_r = data_r[1:]

    # plot the free memory with an area chart
    fig_memory = plt.figure(title=title, animation_duration=500, legend_location=legend_location)
    # ----------------------------------------------------------------------------------------------------
    axes_left_options = {
        "x": {"label": "time", "grid_lines": "solid"},
        "y": {
            "side": "left",
            "grid_lines": "solid",
            "color": "green",
            "label_color": "black",
        },  # "label": "Memory in bytes",
    }
    axes_right_options = {
        "x": {"label": "time", "grid_lines": "solid"},
        "y": {"side": "right", "color": "blue", "label_color": "black"},
    }
    # plot the free memory
    line_1 = plt.plot(
        data_l,
        labels=labels_l,
        display_legend=True,
        colors=["green", "red"],
        axes_options=axes_left_options,
    )

    if name_step > 0:
        # add the log names as labels to the x axis for every n-th entry
        names = [i.name for i in mi_list]
        name_labels = plt.label(
            text=[x.name for x in mi_list][::name_step],
            x=range(len(mi_list))[::name_step],
            y=[y + 10 for y in free_mem[::name_step]],
            default_size=14,
            labels=["name"],
            display_legend=False,
        )
        name_labels.rotate_angle = -30

    # ----------------------------------------------------------------------------------------------------
    if data_r.shape[0] > 0:
        # create a seperate scale for the y axis on the right-hand side of the plot to allow for the difference in scale
        plt.scales(scales={"x": plt.Keep})

        line_3 = plt.plot(
            data_r,
            labels=labels_r,
            display_legend=True,
            axes_options=axes_right_options,
        )
    # code to add a legend to the plot
    # ref:  bqplot.pyplot.show()
    if display_toolbar:
        if not hasattr(fig_memory, "pyplot"):
            fig_memory.pyplot = plt.Toolbar(figure=fig_memory)
            fig_memory.pyplot_vbox = plt.VBox([fig_memory, fig_memory.pyplot])
        return fig_memory.pyplot_vbox
    else:
        return fig_memory

In [22]:
f = meminfo_plot(
    mi_list,
    title="createstubs memory use",
    used=True,
    one_blocks=True,
    two_blocks=True,
    max_block_size=True,
    max_free_size=True,
    stack_used=True,
    stack_total=True,
    name_step=0,
)

ValueError: fig_memory.pyplot not defined

In [20]:
f

### Optional HTML width adjustment to allow wider plots and other outputs


In [ ]:
# use the whole page width for displaying the output in the Jupyter notebook web viewer

from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))